In [58]:
###Exploring the actual circuit implementations for simulation of NMR spectra

import openfermion as of

import sys
sys.path.append('./utils/')

from basis_utils import Sz,Sx,Sz, Sy

import cirq
import numpy as np

In [94]:
def ExpZstring(qub_reg,qub_idxs,angle):
    """
    Implementation of a exp(-i(\theta/2)*Z_idx1 * Z_idx2 * ... * Z_idxN), where idxs represent the indices within the qubit register over which the circuit is implemented 
    """

    #It is assmued that indices are sorted from smallest to largest...

    circuit = cirq.Circuit()
    Length = len(qub_idxs)

    for i in range(Length-1):
        circuit.append(cirq.CNOT(qub_reg[qub_idxs[i]], qub_reg[qub_idxs[i+1]]))

    circuit.append(cirq.rz(angle)(qub_reg[qub_idxs[Length-1]]))

    #reversing CNOTS:
    for i in range(Length-1):
        circuit.append(cirq.CNOT(qub_reg[qub_idxs[Length-2-i]], qub_reg[qub_idxs[Length-1-i]]))

    return circuit

def ExpYstring(qub_reg,qub_idxs,angle):

    circuit = cirq.Circuit()
    Length = len(qub_idxs)
    
    for i in range(Length):
        circuit.append(cirq.rx(np.pi/2)(qub_reg[qub_idxs[i]]))
    
    circ_ph = ExpZstring(qub_reg,qub_idxs,angle)

    circuit.append(circ_ph)

    for i in range(Length):
        circuit.append(cirq.rx(-np.pi/2)(qub_reg[qub_idxs[i]]))

    return circuit


def ExpXstring(qub_reg,qub_idxs,angle):

    circuit = cirq.Circuit()
    Length = len(qub_idxs)
    
    for i in range(Length):
        circuit.append(cirq.ry(np.pi/2)(qub_reg[qub_idxs[i]]))
    
    circ_ph = ExpZstring(qub_reg,qub_idxs,angle)

    circuit.append(circ_ph)

    for i in range(Length):
        circuit.append(cirq.ry(-np.pi/2)(qub_reg[qub_idxs[i]]))

    return circuit


def Heis_int_circ(qub_reg,qub_idxs,angle):

    

    circuit = cirq.Circuit()

    #splitting the interaction into ZZ, YY and XX:
    ZZ = ExpZstring(qub_reg,qub_idxs,angle)
    YY = ExpYstring(qub_reg,qub_idxs,angle)
    XX = ExpXstring(qub_reg,qub_idxs,angle)

    #print(YY)
    circuit.append(ZZ)
    circuit.append(YY)
    circuit.append(XX)

    
    return circuit

def sing_anc_jump_op(x_idx,z_idxs,qub_reg,angle):
    """
    Args:
    x_idx, integer that denotes the index of the X operator
    z_idxs, array of 2 indices, that account for the jump operator 
    """
    
    circuit = cirq.Circuit()

    circuit.append(cirq.H(qub_reg[x_idx]))

    tot_idxs = np.concatenate((np.array([x_idx]),z_idxs))
    tot_idxs = np.sort(tot_idxs)

    circ_ph = ExpZstring(qub_reg,tot_idxs,angle)

    circuit.append(circ_ph)

    circuit.append(cirq.H(qub_reg[x_idx]))

    return circuit


def ZstrXstr(z_idxs,x_idxs,qub_reg,angle):
    
    circuit = cirq.Circuit()
    x_length = len(x_idxs)

    #we combine and sort the z_idxs and x_idxs
    tot_idxs = np.concatenate((z_idxs,x_idxs))
    tot_idxs = np.sort(tot_idxs)
    
    for i in range(x_length):
        circuit.append(cirq.ry(np.pi/2)(qub_reg[x_idxs[i]]))
    
    circ_ph = ExpZstring(qub_reg,tot_idxs,angle)

    circuit.append(circ_ph)

    for i in range(x_length):
        circuit.append(cirq.ry(-np.pi/2)(qub_reg[x_idxs[i]]))

    return circuit

def ZstrYstr(z_idxs,y_idxs,qub_reg,angle):
    
    circuit = cirq.Circuit()
    x_length = len(y_idxs)

    #we combine and sort the z_idxs and x_idxs
    tot_idxs = np.concatenate((z_idxs,y_idxs))
    tot_idxs = np.sort(tot_idxs)
    
    for i in range(x_length):
        circuit.append(cirq.ry(np.pi/2)(qub_reg[y_idxs[i]]))
    
    circ_ph = ExpZstring(qub_reg,tot_idxs,angle)

    circuit.append(circ_ph)

    for i in range(x_length):
        circuit.append(cirq.ry(-np.pi/2)(qub_reg[y_idxs[i]]))

    return circuit



def ZstringHeisInt(z_idxs,sys_idxs,qub_reg,angle):

    #using indexes of "system" for XX term:
    circuit = cirq.Circuit()

    Zs_XX = ZstrXstr(z_idxs,sys_idxs,qub_reg,angle)
    Zs_YY = ZstrYstr(z_idxs,sys_idxs,qub_reg,angle)

    tot_idxs = np.concatenate((z_idxs,sys_idxs))
    tot_idxs = np.sort(tot_idxs)

    Zs_ZZ = ExpZstring(qub_reg,tot_idxs,angle)

    circuit.append(Zs_ZZ)
    circuit.append(Zs_YY)
    circuit.append(Zs_XX)

    return circuit





In [95]:
###Constructing DFG circuit "by hand"1

#TODO: change these parameters for the corresponding ones later

dt = 0.1
sqrtdt= np.sqrt(dt)
w1 = 10.0
w2 = 5.0
ws = [w1,w2]
J12 = 0.2

J_mat = np.zeros([2,2])
J_mat[0,1] = J12

####Construction  of each of the layers of circuit for a qdrift implementation...

###Linear qubit register...
qubs = cirq.LineQubit.range(3)

###These correspond to the \omega S_{z} terms....
ListWZs = []
for i in range(2):
    ListWZs.append(cirq.rz(sqrtdt*ws[i]/2)(qubs[i]))

###General form to construct the circuit for Heisenberg interactions...
ListInts= []
for i in range(2):
    for j in range(i+1,2):
        Heis_int_circ(qubs,np.array([i,j]),J_mat[i,j]*sqrtdt/2.0)







In [96]:

ListWZs

[cirq.Rz(rads=1.5811388300841898).on(cirq.LineQubit(0)),
 cirq.Rz(rads=0.7905694150420949).on(cirq.LineQubit(1))]

In [97]:
ListInts

[]

In [87]:
qubs = cirq.LineQubit.range(3)
circuit = cirq.Circuit()

#qub_idxs = [0,1]#
z_idxs = np.array([2],dtype=int)
sys_idxs = np.array([0,1],dtype=int)
angle=1.0


#Heis_int_circ(qubs,qub_idxs,angle)

#ExpZstring(qubs,qub_idxs,angle)
#ZstringHeisInt(z_idxs,sys_idxs,qubs,angle)
sing_anc_jump_op(2,sys_idxs,qubs,angle)

0: ───────@────────────────────────@───
          │                        │
1: ───────X───@────────────────@───X───
              │                │
2: ───H───────X───Rz(0.318π)───X───H───

In [78]:
sys_idxs

array([0, 1])

In [77]:
np.concatenate(z_idxs,sys_idxs)

TypeError: only integer scalar arrays can be converted to a scalar index

In [33]:
qubs = cirq.LineQubit.range(7)
circuit = cirq.Circuit()

In [34]:
test = cirq.rz(2)
#circuit.append(test.on(qubs[1]))
circuit.append(cirq.rz(2)(qubs[0]))

In [36]:
2/np.pi

0.6366197723675814

In [35]:
circuit

0: ───Rz(0.637π)───

In [30]:
circuit

0: ───Rz(0.25π)────

1: ───Rz(0.637π)───

In [27]:
import cirq
import numpy as np

# Create a qubit
qubit = cirq.LineQubit(0)

# Initialize a circuit
circuit = cirq.Circuit()

# Apply an Rz rotation with a specific angle (e.g., theta = π/4)
theta = np.pi / 4
circuit.append(cirq.rz(theta)(qubit))

# Print the circuit
print(circuit)


0: ───Rz(0.25π)───
